In [1]:
import numpy as np 
import matplotlib.pyplot as plt

In [24]:
# PHYSICAL CONSTANTS 
T = 298 # Temperature [K]
F = 96485 # Faraday’s constant [C/mol]
R = 8.3145 # Gas constant [(C*V)/(mol*K)]
f = F/(R*T) # Reduced electrochemical parameter

# INDEPENDENT VARIABLES 
C = 1.0 #  initial concentration of Ox
D = 1E-6 # Diffusion coefficient [cm^2/s] {default: 1E-6}
eta0 = 0.2 # initial overpotential (relative to redox) [V] {default: 0.2}
etaF = -0.2 # final overpotential (relative to redox) [V] {default: -0.2}
v = 1E-3 # Scan rate [V/s] {default: 1E-3}
n = 1.0  # Number of electrons transferred during reaction {default: 1}
alpha = 0.5  # Transfer coefficient (unitless) {default: 0.5}
k0 = 1E-1 # Heterogeneous rate constant [m/s] {default: 1E-1}
kc = 1E-3 # Chemical rate constant [m/s] {default: 1E-3}

# SIMULATION VARIABLES 
L = 500 # Number of time steps {default: 500}
DM = 0.45 # model diffusion coefficient from problem. Default = 0.45

# DERIVED CONSTANTS 
k = np.array(list(range(0,L))) # time index vector
total_t  = 2*(eta0-etaF)/v # Maximum t value to simulate (1 CV)
del_t  = total_t/L # Time interval
t = del_t * k # Discretized t
del_x  = np.sqrt(D*del_t/DM) # x interval
npts_x = 100  # Number of mesh points {default: 100}

# # REVERSIBILITY PARAMETERS %%
ktk = kc*total_t   # Dimensionless kinetic parameter 
km = ktk/L # Normalized Dimensionless Kinetic parameter )
Lambda = k0/(D*f*v)**(0.5)  # Dimensionless Reversibility Parameter 


In [53]:
# PRE-INITIALIZATION 
C = C / 1000 # Convert C from mol/L to mol/cm3
eta1 = list(eta0 - v * t) # overpotential vector, negative scan
eta2 = list(etaF + v * t) # overpotential vector, positive scan
eta = np.asarray(eta1 + eta2) # Overpotential scane [V] 

Enorm = eta * f # normalized overpotential
kf = k0 *np.exp(-alpha * n * Enorm) # [=] cm/s, fwd rate constant (pg 799)
kb = k0 *np.exp((1-alpha) * n * Enorm) # [=] cm/s, rev rate constant (pg 799)

O = C*np.ones((L+1,npts_x)) # [=] mol/cm^3, concentration of O
R = np.zeros((L+1,npts_x))  # [=] mol/cm^3, concentration of R
Flux = np.zeros((1,L+1)) # [=] mol/cm^2-s, flux of O at the surface

In [55]:
# START SIMULATION 
# i1 = time index. i2 = distance index
for i1 in range (0,L):
    # Update bulk concentrations of O and R
    for i2 in range(1,npts_x-1):
        O[i1+1,i2] = O[i1,i2] + DM*(O[i1,i2+1]+O[i1,i2-1]-2*O[i1,i2])
        R[i1+1,i2] = R[i1,i2] + DM*(R[i1,i2+1]+R[i1,i2-1]-2*R[i1,i2]) - km * R[i1,i2]

    if i1 < L:
        # Update flux
        Flux[i1+1] = (kf[i1+1] *O[i1+1,2] - kb[i1+1] * R[i1+1,2]) / (1 + del_x/D*(kf[i1+1] + kb[i1+1]))

    # Update surface concentrations
    O[i1+1,0] = O[i1+1,2] - JO[i1+1]*(Dx/D);
    R[i1+1,0] = R[i1+1,2] + JO[i1+1]*(Dx/D) - km*R[i1+1,1]

# Calculate current density, Z, from flux of O
Z = -n *F * Flux * 1000; # [=] A/cm^2 -> mA/cm^2, current density


IndexError: index 1 is out of bounds for axis 0 with size 1